In [1]:
pip install deepface

SyntaxError: invalid syntax (<ipython-input-1-b943716737cd>, line 1)

In [ ]:
import pandas as pd
import numpy as np
import itertools
from sklearn.metrics import confusion_matrix,accuracy_score, roc_curve, auc
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

In [2]:
idendities = {
  "Ariel_Sharon": [],
  "Colin_Powell": [],
  "Donald_Rumsfeld": [],
  "George_W_Bush": [],
  "Gerhard_Schroeder": [],
  "Hugo_Chavez": [],
  "Jean_Chretien": [],
  "John_Ashcroft": [],
  "Junichiro_Koizumi": [],
  "Tony_Blair": []
    
}

In [3]:
import os
from PIL import Image

for key, values in idendities.items():
    # Get the path to all the images
        for image in os.listdir("Dataset/DeepFace/{}".format(key)):
            path_string = os.path.join("Dataset/DeepFace/{}".format(key), image)
            image = Image.open(path_string)
            imgNp = np.array(image, "uint8")
            values.append(path_string)


NameError: name 'np' is not defined

In [ ]:
idendities["George_W_Bush"]

<h1>Positive samples<h1>
Find different photos of same people


In [ ]:
positives = []

for key, values in idendities.items():
    
    #print(key)
    for i in range(0, len(values)-1):
        for j in range(i+1, len(values)):
            #print(values[i], " and ", values[j])
            positive = []
            positive.append(values[i])
            positive.append(values[j])
            positives.append(positive)

In [ ]:
positives = pd.DataFrame(positives, columns = ["file_x", "file_y"])
positives["decision"] = "Yes"

In [ ]:
positives.shape


# Negative samples
Compare photos of different people

In [ ]:
samples_list = list(idendities.values())


In [ ]:
negatives = []

for i in range(0, len(idendities) - 1):
    for j in range(i+1, len(idendities)):
        #print(samples_list[i], " vs ",samples_list[j]) 
        cross_product = itertools.product(samples_list[i], samples_list[j])
        cross_product = list(cross_product)
        #print(cross_product)
        
        for cross_sample in cross_product:
            #print(cross_sample[0], " vs ", cross_sample[1])
            negative = []
            negative.append(cross_sample[0])
            negative.append(cross_sample[1])
            negatives.append(negative)

In [ ]:
negatives = pd.DataFrame(negatives, columns = ["file_x", "file_y"])
negatives["decision"] = "No"

In [ ]:
negatives = negatives.sample(positives.shape[0])


In [ ]:
negatives.shape


# Merge Positives and Negative Samples¶


In [ ]:
df = pd.concat([positives, negatives]).reset_index(drop = True)


In [ ]:
df.shape


In [ ]:
df.decision.value_counts()


In [ ]:
df = pd.concat([positives, negatives]).reset_index(drop = True)

In [ ]:
df.decision

# DeepFace

In [ ]:
from deepface import DeepFace
from deepface.basemodels import VGGFace, OpenFace, Facenet, FbDeepFace

In [ ]:
pretrained_models = {}

pretrained_models["VGG-Face"] = VGGFace.loadModel()
print("VGG-Face loaded")

pretrained_models["Facenet"] = Facenet.loadModel()
print("Facenet loaded")

# pretrained_models["OpenFace"] = OpenFace.loadModel() 
# print("OpenFace loaded")

pretrained_models["DeepFace"] = FbDeepFace.loadModel()
print("FbDeepFace loaded")

In [ ]:
instances = df[["file_x", "file_y"]].values.tolist()


In [ ]:
models = ['VGG-Face', 'Facenet', 'DeepFace']
metrics = ['cosine', 'euclidean', 'euclidean_l2']

In [ ]:
if True:
    for model in models:
        for metric in metrics:
            
            print("Processing ",model," ",metric)
            
            if model == 'OpenFace' and metric == 'euclidean': #this returns same with openface euclidean l2
                continue
            else:
                resp_obj = DeepFace.verify(instances
                                           , model_name = model
                                           , model = pretrained_models[model]
                                           , distance_metric = metric
                                           , detector_backend = 'mtcnn'
                                           , enforce_detection = False)

                distances = []

                for i in range(0, len(instances)):
                    distance = round(resp_obj["pair_%s" % (i+1)]["distance"], 4)
                    distances.append(distance)

                df['%s_%s' % (model, metric)] = distances
    
    df.to_csv("face-recognition-pivot.csv", index = False)
else:
    #Ref: https://github.com/serengil/deepface/blob/master/tests/dataset/face-recognition-pivot.csv
    df = pd.read_csv("face-recognition-pivot.csv")

In [ ]:
df_raw = df.copy()


In [ ]:
df.head()


# Distribution

In [ ]:
df = pd.read_csv("face-recognition-pivot.csv")

In [ ]:
df_raw = df.copy()


In [ ]:
df.head()

In [ ]:


fig = plt.figure(figsize=(15, 15))

figure_idx = 1
for model in models:
    for metric in metrics:
        
        
        feature = '%s_%s' % (model, metric)

        ax1 = fig.add_subplot(4, 3, figure_idx)
        
        df[df.decision == "Yes"][feature].plot(kind='kde', title = feature, label = 'Yes', legend = True)
        df[df.decision == "No"][feature].plot(kind='kde', title = feature, label = 'No', legend = True)
        
        figure_idx = figure_idx + 1

plt.show()

# Pre-processing for model

In [ ]:
columns = []
for model in models:
    for metric in metrics:
        if model == 'OpenFace' and metric == 'euclidean':
            continue
        else:
            feature = '%s_%s' % (model, metric)
            columns.append(feature)

columns.append("decision")

In [ ]:
df = df[columns]


In [ ]:

df.loc[df[df.decision == 'Yes'].index, 'decision'] = 1
df.loc[df[df.decision == 'No'].index, 'decision'] = 0

In [ ]:
df.head()


# Train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=34)


In [ ]:
target_name = "decision"

y_train = df_train[target_name].values
x_train = df_train.drop(columns=[target_name]).values

y_test = df_test[target_name].values
x_test = df_test.drop(columns=[target_name]).values

# lightGBM


In [ ]:
!pip install lightgbm

In [ ]:
import lightgbm as lgb


In [ ]:
features = df.drop(columns=[target_name]).columns.tolist()

lgb_train = lgb.Dataset(x_train, y_train, feature_name = features)
lgb_test = lgb.Dataset(x_test, y_test, feature_name = features)

In [ ]:
params = {
    'task': 'train'
    , 'boosting_type': 'gbdt'
    , 'objective': 'multiclass'
    , 'num_class': 2
    , 'metric': 'multi_logloss'
    , 'force_col_wise': 'true'
}

In [ ]:
gbm = lgb.train(params, lgb_train, num_boost_round=400, early_stopping_rounds = 20 , valid_sets=[lgb_test])
# gbm = lgb.train(params, lgb_train, num_boost_round=400, valid_sets=[lgb_test])


In [ ]:
gbm.save_model("face-recognition-ensemble-model_test30.txt")


# Evaluation

In [ ]:
predictions = gbm.predict(x_test)


In [ ]:
prediction_classes = []
classified = 0

index = 0
for prediction in predictions:
    prediction_class = np.argmax(prediction)
    prediction_classes.append(prediction_class)
    
    actual = y_test[index]
    
    print("prediction is ",prediction_class," whereas actual is ",actual)
    if actual == prediction_class:
        classified = classified + 1
    
    index = index + 1

#print(classified," instances are classified in ",len(predictions)," instances") 
print("accuracy: ",round(100*classified/len(predictions),2),"%")

In [ ]:
print(classified," instances are classified in ",len(predictions)," instances") 


In [ ]:
print(y_test), print(prediction_classes)

In [ ]:
prediction_classes_test = np.array(prediction_classes) 
y_test_arr = y_test


print(y_test_arr.tolist()), print(prediction_classes)

In [ ]:
cm = confusion_matrix(y_test_arr.tolist(), prediction_classes_test)

In [ ]:
cm

In [ ]:
tn, fp, fn, tp = cm.ravel()


In [ ]:
tn, fp, fn, tp


In [ ]:
recall = tp / (tp + fn)
precision = tp / (tp + fp)
accuracy = (tp + tn)/(tn + fp +  fn + tp)
f1 = 2 * (precision * recall) / (precision + recall)

In [ ]:
print("Precision: ", 100*precision,"%")
print("Recall: ", 100*recall,"%")
print("F1 score ",100*f1, "%")
print("Accuracy: ", 100*accuracy,"%")

In [ ]:
plt.figure(figsize=(7,7))
ax = lgb.plot_importance(gbm, max_num_features=20)
plt.show()

In [ ]:
y_pred_proba = predictions[::,1]


In [ ]:
from sklearn import metrics
fpr, tpr, _ = metrics.roc_curve(y_test_arr.tolist(),  y_pred_proba)
auc = metrics.roc_auc_score(y_test_arr.tolist(), y_pred_proba)

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))

In [ ]:
deepface_ensemble = lgb.Booster(model_file= 'face-recognition-ensemble-model_test30.txt')


In [ ]:
from PIL import Image
testImage = Image.open("Test_img/demo1.jpg")
testimgNp = np.array(testImage, "uint8")

In [ ]:
bulk_predictions = deepface_ensemble.predict("Test_img/demo1.jpg")
print(bulk_predictions)

In [ ]:
demo = lgb.Booster(model_file= 'Train/face-recognition-ensemble-model.txt')


In [ ]:
demo.predict(x_test)

In [ ]:
x_test